In [ ]:
pip install -q yahoo_fin

In [ ]:
pip install --upgrade yfinance

In [3]:
mkdir -p data results log

In [4]:
from google.colab import drive
drive.mount('/content/drive')
! find /content/drive/MyDrive/colab/ -type f -mtime +180 -delete -print


Mounted at /content/drive


In [5]:
import numpy as np
import pandas as pd

In [9]:
def show_prediction(name, suffix):
  df = pd.read_csv(f'results/{name}_{suffix}_predictions.csv')
  df['predicted_return'] = ((df['predicted_max']+df['predicted_min']+df['predicted_close'])/3 - df['current_price'])/df['current_price']
  return df.sort_values('predicted_return', ascending=False, key=abs)


In [6]:
def run_command_with_check(command):
  """Runs a command, prints stderr, and raises an exception if it fails."""
  process = subprocess.run(command, shell=True, capture_output=True, text=True, check=False)

  if process.returncode != 0:
    print("stderr:", process.stderr)  # Print stderr only on error
    raise subprocess.CalledProcessError(process.returncode, process.args, output=process.stdout, stderr=process.stderr)
  else:
    # Print or process stdout if needed
    #print("stdout:", process.stdout)
    pass

  return process.returncode


In [7]:
import os
import subprocess
def run_model_nn(prefix, version, do_train=False, window_size=None, future_days=None, hourly=False):
    # copy drive/MyDrive/colab/results/nnreturn.py to local folder
    tickers = f"{prefix}tickers.csv"
    run_command_with_check(f"cp  /content/drive/MyDrive/colab/nnreturn.py .")
    run_command_with_check(f"cp  /content/drive/MyDrive/colab/tickers/{tickers} .")
    if not do_train:
      run_command_with_check(f"cp  /content/drive/MyDrive/colab/models/{prefix}_{version}_*.pth .")
      # Run nnreturn.py in results folder and throw exception if error
      today = pd.to_datetime('today').strftime('%m%d')
      run_command_with_check(f"rename 's/{version}/{today}/' *.pth")
      cmd_arg = ""
      if window_size:
        cmd_arg += f" --window-size {window_size}"
      if future_days:
        cmd_arg += f" --future-days {future_days}"
      if hourly:
        cmd_arg += " --hourly"
      run_command_with_check(f"python3 nnreturn.py --tickers-file {prefix}tickers.csv --mode analyze {cmd_arg}")
      run_command_with_check(f"cp {prefix}*predictions.csv results")
      # Get today's date

      return show_prediction(prefix, today)
    else:
      raise Exception("Not implemented")


In [ ]:
# 0716 is hourly 20
run_model_nn('idx', '0716', window_size=20, future_days=8, hourly=True).head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
23,XLB,89.61,2025-07-17,2025-07-19,89.62,90.66,91.40,0.010601
31,XLV,132.58,2025-07-17,2025-07-19,132.34,134.13,134.68,0.008573
20,URA,40.29,2025-07-17,2025-07-19,39.80,40.24,39.87,-0.007942
4,EEM,49.12,2025-07-17,2025-07-19,48.63,48.96,48.71,-0.007193
9,IXP,110.10,2025-07-17,2025-07-19,109.92,111.52,110.81,0.005904
14,JXI,73.21,2025-07-17,2025-07-19,72.43,73.40,72.53,-0.005782
18,SMH,292.19,2025-07-17,2025-07-19,292.37,295.67,293.22,0.005350
13,JETS,25.00,2025-07-17,2025-07-19,24.92,25.34,25.07,0.004400
11,IWO,293.85,2025-07-17,2025-07-19,292.41,296.41,296.18,0.003914
12,IYZ,29.77,2025-07-17,2025-07-19,29.50,29.77,29.72,-0.003583


In [ ]:
run_model_nn('etf', '0202').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
25,ONLN,39.53,2025-04-17,2025-05-15,38.56,43.89,41.66,0.046547
31,URA,22.87,2025-04-17,2025-05-15,22.95,24.86,23.32,0.036729
39,XLC,90.21,2025-04-17,2025-05-15,88.96,94.82,95.64,0.032480
48,XLY,185.40,2025-04-17,2025-05-15,182.42,196.79,194.70,0.031841
0,ARKF,32.56,2025-04-17,2025-05-15,32.04,35.15,33.46,0.030405
9,FDN,211.93,2025-04-17,2025-05-15,205.40,231.13,218.53,0.030309
5,DIA,391.36,2025-04-17,2025-05-15,382.35,418.51,407.83,0.029478
50,XNTK,177.86,2025-04-17,2025-05-15,174.22,194.06,180.70,0.028862
20,JETS,18.88,2025-04-17,2025-05-15,18.87,20.09,19.29,0.028425
35,VUG,352.14,2025-04-17,2025-05-15,342.50,378.89,364.62,0.028010


In [ ]:
# pip install  git+https://github.com/ahsank/runml#egg=runml

In [ ]:
import runml.pipeline
from runml import pipeline,findata
#from importlib import reload
# reload(runml.pipeline)

In [ ]:
# Skip some days from consecutive window to lessen test /train overlap
# findata.G_NON_OVERLAP= 0

Run following if like to save results in drive

In [ ]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    # Simplify following using apply
    def calculate_alloc(row):
        gain_low = min(row['Gain_l'], 0)
        gain_high = max(row['Gain_h'], 0)
        gain = min(row['Gain'], gain_high)
        average_gain = (gain_low+gain_high + 2*gain)/4
        if (average_gain == 0):
          return 0
        avg_accuracy = (row['Accu']+row['Accu_l']+row['Accu_h'])/3
        alloc =  avg_accuracy/stop_loss - (1-avg_accuracy)*profit_factor/abs(average_gain)
        return round(max(alloc, 0), 2)
    df['Alloc'] = df.apply(calculate_alloc, axis=1)
    return df

pd.options.display.max_columns = None

In [ ]:
import os
# findata.EPOCHS=200
def run_model(prefix, version, tickers, do_train):
  if (not do_train):
    os.system(f"cp  /content/drive/MyDrive/colab/results/{prefix}-{version}-* ./results/")
  mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(),
                         pipeline.AddMA(200), pipeline.Adj()]))
  df= pipeline.runModelCombinedVola(tickers, f"{prefix}-{version}", mod, do_train)
  # Save datafrme to prefix-version-prediction.csv
  df.to_csv(f"results/{prefix}-{version}-prediction.csv")
  return df



In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'minmax' # Change to minmax
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BASE', 'BILL', 'BMBL', 'BYRN', 'CELH', 'CFLT', 'COHR', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ELF', 'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'HCP', 'HIMS', 'HOOD', 'HUBS',
           'INMD', 'INTA', 'IOT', 'IRDM', 'JAKK', 'JKS', 'KD', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR', 'MRVL', 'MXL','MTCH', 'MU',
           'NET', 'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PAR', 'PANW',  'PAYX', 'PD', 'PI', 'PINS',
           'RBLX', 'RKLB', 'RMBS', 'RUM', 'SMCI', 'SMAR', 'SNAP', 'SNOW', 'SQ', 'SPOT',
           'SOUN', 'TEM', 'TEAM', 'TDOC', 'TMDX', 'TNDM', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
df = run_model('ipos', '2a', tickers, True)

In [ ]:
addAllocHL(df, 0.10, 1)
df[(df.Gain>0)].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'CIBR', 'DAPP', 'DIA', 'DTEC', 'DXYZ',
            'EEM', 'FDN', 'FPX',
            'ICLN', 'IGV', 'IJK', 'IJR', 'IPO',
            'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'JXI', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'URA', 'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
df1 = run_model('etf', '1b', tickers1, True)



In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

In [ ]:
df1.to_csv('results/etf_12b_predictions.csv')

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'minmax'
tickers3 = [ 'AIRI', 'ALSMY', 'AMSC', 'AMSSY', 'ACHR',
            'APPS',  'AUR',  'ATOM', 'AWRE', 'AXTI',
            'BKKT', 'BGSF', 'BLDE', 'BLNK',
            'CARS', 'CHGG', 'CLOV', 'CRCT', 'CXM',
            'DLO', 'DM', 'EVGO', 'FSLY',
            'GOGO', 'GRAB', 'GRPN', 'GTE', 'HIMX', 'HIVE',
            'INDI', 'INLX',  'INVZ', 'JOBY',  'KIND',
            'LAZR', 'LFMD', 'LUMN', 'MAX', 'MITK', 'MTTR', 'MYTE',
            'NEPH', 'NNOX', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PGY', 'PLUG', 'PTON', 'PUBM',
            'REAL', 'REI', 'RIG', 'RVYL',
            'SABR', 'SFIX', 'SLDP', 'SLE', 'SMWB', 'SST', 'STEM',
            'TDW', 'TDOC', 'TRUP', 'VTEX', 'VSAT', 'YEXT']
df3 = run_model('vols', '12b', tickers3, False)

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3[(df3.Gain>0)].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DELL', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG', 'KO', 'LLY',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX',
            'TMO', 'TSLA', 'TXN', 'UNH', 'UPS', 'WMT']
df4 = run_model('mcap', '1b', tickers4, True)

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
tickers5 = ['ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CX', 'DBOEY', 'DNNGY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FDRVF', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LIN', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']

df5 = run_model('gstock', '2a', tickers5, False)

In [ ]:
addAllocHL(df5, 0.14)
df5[df5.Gain > 0].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.G_SCALER = 'standard'
tickers6 = ['ARGT', 'EWD', 'FLN',
           'ECH', 'EPHE', 'EWA', 'EWG', 'EWI', 'EWJ',
           'EWC', 'EWM', 'EWP', 'EWT', 'EWW', 'EWY', 'EWZ',
           'IDX', 'ILF', 'INDA', 'KEMQ', 'KWT', 'THD', 'TUR', 'VNM', 'XCEM']
df6 = run_model('etf', '12a', tickers6, False)

In [ ]:
addAllocHL(df6, 0.07, 1)
df6.sort_values('Alloc', ascending=False)

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'standard'
tickers7 = ['AAL', 'ADI', 'ALB', 'AMR','ANF', 'APO', 'AQN', 'ARCH', 'ARE', 'ASC',
            'BAH', 'BAX', 'BKR', 'BLK', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IFF', 'IP', 'IPI', 'IVZ',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SBLK', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW', 'ZTS']
df7 = run_model('val', '1a', tickers7, False)

In [ ]:
addAllocHL(df7, 0.10, 1)
df7[df7.Gain > 0].sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
tickers8 = [ 'DBC', 'FXB', 'FXE', 'FXF', 'FXY',
            'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'USO', 'UUP', 'VTIP', 'WEAT']

df8 =  run_model('com', '12a', tickers8, True)

In [ ]:
addAllocHL(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
tickers9 = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'BCH-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'FIL-USD', 'ICP-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'NEAR-USD', 'SOL-USD', 'TON-USD', 'TRX-USD', 'XRP-USD']
df9 = run_model('cry', '11b', tickers9, False)

In [ ]:
addAllocHL(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

In [ ]:
run_model_nn('pennystock', '0630').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
62,SST,5.55,2025-06-30,2025-07-28,5.22,6.99,8.53,0.245646
10,BKKT,13.92,2025-06-30,2025-07-28,13.47,21.68,16.06,0.226293
31,INVZ,1.64,2025-06-30,2025-07-28,1.19,1.62,1.36,-0.152439
42,NEPH,4.08,2025-06-30,2025-07-28,3.26,4.22,3.25,-0.123366
11,BGSF,6.35,2025-06-30,2025-07-28,4.65,6.39,5.69,-0.121785
26,GTE,4.78,2025-06-30,2025-07-28,3.27,4.94,4.66,-0.102510
34,KIND,1.65,2025-06-30,2025-07-28,1.58,1.95,1.89,0.094949
59,SLDP,2.19,2025-06-30,2025-07-28,1.55,2.42,1.98,-0.094368
56,RVYL,0.85,2025-06-30,2025-07-28,0.80,0.99,1.00,0.094118
36,LFMD,13.62,2025-06-30,2025-07-28,10.67,14.15,12.37,-0.089819


In [ ]:
run_model_nn('bluechipstock', '0221').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
19,FDX,215.88,2025-04-23,2025-05-21,212.46,242.33,233.78,0.063199
34,QCOM,142.90,2025-04-23,2025-05-21,140.66,158.85,153.34,0.056333
1,ACN,294.56,2025-04-23,2025-05-21,291.41,320.95,309.08,0.042730
2,ADBE,357.65,2025-04-23,2025-05-21,355.82,380.70,379.86,0.040477
40,UPS,99.00,2025-04-23,2025-05-21,95.83,104.98,106.85,0.035892
15,DELL,90.61,2025-04-23,2025-05-21,90.18,99.42,91.91,0.035610
33,PYPL,63.65,2025-04-23,2025-05-21,59.74,68.96,68.93,0.034983
26,META,529.75,2025-04-23,2025-05-21,503.05,581.74,556.46,0.032720
30,NVDA,103.62,2025-04-23,2025-05-21,98.30,112.89,109.60,0.031944
24,KO,73.43,2025-04-23,2025-05-21,72.71,79.01,74.46,0.026737


In [ ]:
run_model_nn('country', '0224').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
3,ECH,31.67,2025-04-25,2025-05-23,29.52,32.54,29.63,-0.034944
7,EWI,44.04,2025-04-25,2025-05-23,40.22,45.26,42.10,-0.034363
11,EWP,40.86,2025-04-25,2025-05-23,36.79,42.01,41.18,-0.021211
17,ILF,24.90,2025-04-25,2025-05-23,23.96,25.83,23.37,-0.020616
21,THD,53.11,2025-04-25,2025-05-23,52.15,56.15,54.28,0.020398
6,EWG,39.36,2025-04-25,2025-05-23,37.23,40.48,38.05,-0.019648
4,EPHE,26.26,2025-04-25,2025-05-23,26.09,27.18,26.96,0.018406
9,EWC,42.04,2025-04-25,2025-05-23,41.29,43.43,43.39,0.015779
20,KWT,35.71,2025-04-25,2025-05-23,34.17,36.68,34.69,-0.014842
2,FLN,18.61,2025-04-25,2025-05-23,17.94,19.28,17.79,-0.014687


In [ ]:
run_model_nn('global', '0225').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
8,COTY,4.98,2025-04-25,2025-05-23,4.90,6.00,6.01,0.131861
60,THQQF,11.23,2025-04-24,2025-05-22,11.47,13.28,12.97,0.119620
27,GLOB,118.72,2025-04-25,2025-05-23,118.43,142.37,132.32,0.103774
13,DNNGY,13.05,2025-04-25,2025-05-23,12.81,14.01,14.77,0.062324
34,MGA,34.12,2025-04-25,2025-05-23,32.25,36.00,40.01,0.057640
1,ADDDF,248.70,2025-04-24,2025-05-22,244.89,271.26,269.23,0.052647
28,HCMLY,22.07,2025-04-25,2025-05-23,19.87,22.23,20.70,-0.051503
26,GLCNF,3.55,2025-04-25,2025-05-23,3.48,3.79,3.89,0.047887
35,RACE,458.45,2025-04-25,2025-05-23,450.75,498.41,488.70,0.045450
3,BAK,4.01,2025-04-25,2025-05-23,3.87,4.49,4.19,0.043225


In [ ]:
run_model_nn('valuestock', '0301').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
0,AAL,9.75,2025-04-25,2025-05-23,9.71,11.30,10.93,0.091966
38,FIS,79.26,2025-04-25,2025-05-23,71.36,78.02,68.53,-0.083565
29,DAL,41.58,2025-04-25,2025-05-23,41.67,47.85,45.44,0.081930
3,AMR,126.20,2025-04-25,2025-05-23,123.92,145.58,139.42,0.080085
34,EMN,75.84,2025-04-25,2025-05-23,75.92,90.49,78.07,0.074543
77,SBLK,14.49,2025-04-25,2025-05-23,14.51,16.53,15.49,0.070393
9,BAH,119.59,2025-04-25,2025-05-23,118.64,135.64,129.03,0.068400
23,CG,39.03,2025-04-25,2025-05-23,38.82,43.33,42.88,0.067811
46,HPE,16.24,2025-04-25,2025-05-23,15.89,18.10,17.83,0.063629
74,RH,185.49,2025-04-25,2025-05-23,174.08,206.49,209.80,0.060920


In [ ]:
run_model_nn('commodity', '0312').head(5)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
13,UUP,27.50,2025-10-03,2025-10-31,26.83,28.58,27.69,0.007273
15,WEAT,4.16,2025-10-03,2025-10-31,4.03,4.30,4.20,0.004006
4,FXY,62.41,2025-10-03,2025-10-31,60.78,64.46,62.65,0.003525
8,SLV,43.58,2025-10-03,2025-10-31,42.44,44.67,43.23,-0.003060
0,DBC,22.47,2025-10-03,2025-10-31,21.88,23.05,22.28,-0.002967


In [ ]:
run_model_nn('crypto', '0214').head(5)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
4,XRP-USD,3.05,2025-10-03,2025-10-31,2.43,3.06,2.29,-0.149727
2,SOL-USD,232.42,2025-10-03,2025-10-31,182.86,227.72,190.84,-0.137452
3,ADA-USD,0.87,2025-10-03,2025-10-31,0.68,0.90,0.69,-0.130268
0,BTC-USD,122642.72,2025-10-03,2025-10-31,95009.99,127961.45,101393.98,-0.118400
13,LINK-USD,22.51,2025-10-03,2025-10-31,19.06,22.59,18.59,-0.107952


In [10]:
run_model_nn('techstock', '0312').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
44,MNDY,183.57,2025-10-03,2025-10-31,170.76,262.83,196.62,0.144359
79,UPWK,17.27,2025-10-03,2025-10-31,14.01,18.50,14.06,-0.101139
3,ALAB,200.74,2025-10-03,2025-10-31,162.02,211.47,170.21,-0.097174
14,CVNA,375.72,2025-10-03,2025-10-31,331.65,374.29,312.08,-0.096827
31,GLBE,35.37,2025-10-03,2025-10-31,30.39,35.55,30.67,-0.089530
0,ABNB,120.22,2025-10-03,2025-10-31,115.50,135.91,140.68,0.087146
20,DOCU,69.73,2025-10-03,2025-10-31,67.03,82.43,76.55,0.080405
69,TDOC,9.01,2025-10-03,2025-10-31,8.83,10.42,9.95,0.080281
71,TNDM,14.71,2025-10-03,2025-10-31,13.74,17.17,16.60,0.076592
32,HIMS,52.54,2025-10-03,2025-10-31,44.49,57.39,43.94,-0.074864


In [ ]:
run_model_nn('stock', '0123').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
204,PUBM,12.82,2025-07-03,2025-07-31,12.64,16.44,14.37,0.129745
375,PI,115.86,2025-07-03,2025-07-31,116.02,144.76,128.19,0.119080
17,INTA,48.11,2025-07-03,2025-07-31,45.99,55.07,59.27,0.110857
352,ABR,10.90,2025-07-03,2025-07-31,10.53,12.98,12.60,0.104281
144,LFMD,12.50,2025-07-03,2025-07-31,11.93,14.68,14.55,0.097600
378,RCAT,7.08,2025-07-03,2025-07-31,5.73,7.56,5.93,-0.095104
10,ARQT,13.82,2025-07-03,2025-07-31,13.37,16.73,15.27,0.094308
348,CRM,272.15,2025-07-03,2025-07-31,267.03,319.47,305.57,0.092620
283,NTGR,29.41,2025-07-03,2025-07-31,28.97,35.49,31.90,0.092146
150,POWL,217.66,2025-07-03,2025-07-31,204.27,250.88,256.42,0.089727


In [ ]:
run_model_nn('buystocks', '0402', False, 500, 250).head(10)
# show_prediction('buystocks', '0508').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
360,QBTS,28.93,2025-10-02,2026-09-17,27.59,60.07,54.54,0.638438
101,CHGG,1.58,2025-10-02,2026-09-17,1.56,3.13,3.04,0.630802
18,OUST,28.67,2025-10-02,2026-09-17,27.81,58.04,54.09,0.627020
102,SLP,15.53,2025-10-02,2026-09-17,15.06,30.12,30.30,0.620090
92,SST,7.57,2025-10-02,2026-09-17,7.56,14.85,13.27,0.571114
295,CRNC,13.38,2025-10-02,2026-09-17,13.34,24.51,24.66,0.557299
268,LYFT,21.83,2025-10-02,2026-09-17,20.34,42.81,38.05,0.545274
325,BE,90.07,2025-10-02,2026-09-17,83.55,172.24,158.40,0.532845
357,ACHR,10.14,2025-10-02,2026-09-17,9.56,20.44,16.59,0.531558
9,HIMS,57.68,2025-10-02,2026-09-17,48.57,107.11,109.22,0.530860


In [14]:
import sys
sys.path.append('/content')
import stock_prediction.stock_prediction.stock_predictor_lib
from stock_prediction.stock_prediction.stock_predictor_lib import run_model_nn, show_prediction, StockPredictorLib, Config

In [18]:
techcfg = Config.from_file('./stock_prediction/configs/experiment.yaml')
colab_path = 'content/drive/MyDrive/colab'
techcfg.model_path = f'/{colab_path}/models/transformer/techstock_1005'
techdf = run_model_nn(f'/{colab_path}/tickers/techstocktickers.csv', cfg=techcfg)
techdf.head(10)

Loading models for return types: {'min': '/content/drive/MyDrive/colab/models/transformer/techstock_1005_min_model.pth', 'max': '/content/drive/MyDrive/colab/models/transformer/techstock_1005_max_model.pth', 'close': '/content/drive/MyDrive/colab/models/transformer/techstock_1005_close_model.pth'}


,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
50,NOVT,104.53,2025-10-03,2025-10-31,74.96,108.91,63.86,-0.210019
72,TOST,36.64,2025-10-03,2025-10-31,33.50,52.46,45.95,0.200055
12,COHR,113.58,2025-10-03,2025-10-31,85.62,117.58,72.28,-0.191524
11,CFLT,20.30,2025-10-03,2025-10-31,17.17,25.01,29.39,0.175205
26,ETSY,72.38,2025-10-03,2025-10-31,60.19,90.29,102.37,0.164456
42,MBLY,14.67,2025-10-03,2025-10-31,13.98,19.78,16.85,0.149966
77,UI,662.37,2025-10-03,2025-10-31,589.44,682.75,428.81,-0.143983
59,PI,185.19,2025-10-03,2025-10-31,175.74,226.58,231.73,0.141260
5,APP,682.76,2025-10-03,2025-10-31,650.69,896.25,786.22,0.139083
15,DASH,271.22,2025-10-03,2025-10-31,212.24,280.75,220.97,-0.122533


In [8]:
!pwd

/content


In [ ]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +180 -delete -print

sending incremental file list
buystocks_0709_predictions.csv

sent 18,897 bytes  received 35 bytes  37,864.00 bytes/sec
total size is 18,770  speedup is 0.99
